In [18]:
# -*- coding: utf-8 -*

from sympy import *
from time import time
from mpmath import radians
import Ipynb_importer
import transformations
import numpy as np
#import tf


'''
Format of test case is [ [[EE position],[EE orientation as quaternions]],[WC location],[joint angles]]
You can generate additional test cases by setting up your kuka project and running `$ roslaunch kuka_arm forward_kinematics.launch`
From here you can adjust the joint angles to find thetas, use the gripper to extract positions and orientation (in quaternion xyzw) and lastly use link 5
to find the position of the wrist center. These newly generated test cases can be added to the test_cases dictionary.
'''

test_cases = {1:[[[2.16135,-1.42635,1.55109],
                  [0.708611,0.186356,-0.157931,0.661967]],
                  [1.89451,-1.44302,1.69366],
                  [-0.65,0.45,-0.36,0.95,0.79,0.49]],
              2:[[[-0.56754,0.93663,3.0038],
                  [0.62073, 0.48318,0.38759,0.480629]],
                  [-0.638,0.64198,2.9988],
                  [-0.79,-0.11,-2.33,1.94,1.14,-3.68]],
              3:[[[-1.3863,0.02074,0.90986],
                  [0.01735,-0.2179,0.9025,0.371016]],
                  [-1.1669,-0.17989,0.85137],
                  [-2.99,-0.12,0.94,4.06,1.29,-4.12]],
              4:[],
              5:[]}



def test_code(test_case):
    ## Set up code
    ## Do not modify!
    x = 0
    class Position:
        def __init__(self,EE_pos):
            self.x = EE_pos[0]
            self.y = EE_pos[1]
            self.z = EE_pos[2]
    class Orientation:
        def __init__(self,EE_ori):
            self.x = EE_ori[0]
            self.y = EE_ori[1]
            self.z = EE_ori[2]
            self.w = EE_ori[3]

    position = Position(test_case[0][0])
    orientation = Orientation(test_case[0][1])

    class Combine:
        def __init__(self,position,orientation):
            self.position = position
            self.orientation = orientation

    comb = Combine(position,orientation)

    class Pose:
        def __init__(self,comb):
            self.poses = [comb]

    req = Pose(comb)
    start_time = time()
    
    ########################################################################################
    ## 

    ## Insert IK code here!
    print("BEGIN INVERSE KINEMATICS CALCULATION")
    q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # joint variables of theta 
    d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # link offset d_i = signed distance from X_i-1 to X_i along Z_i
    a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7') # non-zero link length a_i-1 = Z_i-1 to Z_i along X_i-1 where X_i-1 is perpendicular to both Z_i-1 to Z_i 
    alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7') # twist angel alpha_i-1 = Z_i-1 and Z_i measured about X_i-1 in a right-hand sense  
 
    # Create Modified DH parameters
    s = {alpha0: 0,     a0: 0,      d1: 0.75, 
         alpha1: -pi/2, a1: 0.35,   d2: 0,     q2: q2-pi/2,  
         alpha2: 0,     a2: 1.25,   d3: 0,
         alpha3: -pi/2, a3: -0.054, d4: 1.50,
         alpha4: pi/2,  a4: 0,      d5: 0,
         alpha5: -pi/2, a5: 0,      d6: 0,
         alpha6: 0,     a6: 0,      d7: 0.303, q7: 0}     
    
    px = req.poses[x].position.x
    py = req.poses[x].position.y
    pz = req.poses[x].position.z

    (roll, pitch, yaw) = transformations.euler_from_quaternion(
        [req.poses[x].orientation.x, req.poses[x].orientation.y,
            req.poses[x].orientation.z, req.poses[x].orientation.w])
        
    # Define Modified DH Transformation matrix
    def TF_Matrix(alpha, a, d, q):
        TF = Matrix([[             cos(q),            -sin(q),            0,              a],
                     [ sin(q)*cos(alpha), cos(q)*cos(alpha), -sin(alpha), -sin(alpha)*d],
                     [ sin(q)*sin(alpha), cos(q)*sin(alpha),  cos(alpha),  cos(alpha)*d],
                     [                   0,                   0,            0,               1]])
        return TF
    
    # Create individual transformation matrices
    T0_1 = TF_Matrix(alpha0, a0, d1, q1).subs(s)
    T1_2 = TF_Matrix(alpha1, a1, d2, q2).subs(s)
    T2_3 = TF_Matrix(alpha2, a2, d3, q3).subs(s)
    T3_4 = TF_Matrix(alpha3, a3, d4, q4).subs(s)
    T4_5 = TF_Matrix(alpha4, a4, d5, q5).subs(s)
    T5_6 = TF_Matrix(alpha5, a5, d6, q6).subs(s)
    T6_G = TF_Matrix(alpha6, a6, d7, q7).subs(s)
    
    T0_3 = T0_1 * T1_2 * T2_3 
    R0_3 = zeros(4,4)
    R0_3[:3, :3] = T0_3[:3, :3]
    R0_3[3, 3] = 1 # perpare for inverse kinematic
       
    T0_G = T0_3 * T3_4 * T4_5 * T5_6 * T6_G # base link to link gripper
 
    # Euler angle calculation
    # Define RPY rotation matrices http://planning.cs.uiuc.edu/node102.html
    #Rot(Z,yaw−alpha)∗Rot(Y,pitch−beta)∗Rot(X,roll−gamma)∗R_corr
    r, p, y = symbols('r p y')
    R_rx = Matrix([[ 1,              0,        0, 0],
                          [ 0,        cos(r), -sin(r), 0],
                          [ 0,        sin(r),  cos(r), 0],
                          [0, 0, 0, 1]])
    R_ry = Matrix([[ cos(p),        0,  sin(p), 0],
                          [       0,        1,        0, 0],
                          [-sin(p),        0,  cos(p), 0],
                          [0, 0, 0, 1]])
    R_rz = Matrix([[ cos(y), -sin(y),        0, 0],
                          [ sin(y),  cos(y),        0, 0],
                          [ 0,              0,        1, 0],
                          [0, 0, 0, 1]])
   
    R_corr = R_rz.subs(y, np.pi) * R_ry.subs(p, -np.pi/2) # The correction for DH coordinate and URDF coordinate
    R_rpy = R_rz * R_ry * R_rx * R_corr
 
    R_rpy = R_rpy.subs({r: roll, p: pitch, y: yaw})     
   
    ### Your IK code here 
    # Calculate joint angles using Geometric IK method
    #d_7 = s[d7] = 0.303
    #d_6 = s[d6] = 0
    wx = px - 0.303 * R_rpy[0,2]
    wy = py - 0.303 * R_rpy[1,2]
    wz = pz - 0.303 * R_rpy[2,2]
           
    #Joint 1, joint2, joint3
    theta1 = atan2(wy, wx)
    # from pic-2, DH table
    a_2 = s[a2] #= 1.25
    d_4 = s[d4] #= 1.50
    a_3 = abs(s[a3]) #= 0.054
    a_1 = s[a1] #= 0.35
    d_1 = s[d1] #= 0.75
    s1 = sqrt(wx*wx+wy*wy) - 0.35
    s2 = wz - 0.75
    s3 = sqrt(pow(s1,2) + pow(s2,2))
    #s4 = sqrt(pow(0.054,2) + pow(1.50,2))
    s4 = 1.501
    beta1 = atan2(s2, s1)
    #beta4 = atan2(0.054, 1.50)
    #beta4 = 0.036
    cosbeta2 = (pow(1.25,2) + pow(s3,2) - pow(s4,2)) / (2 * 1.25 * s3)
    #These formulas produce high round-off errors in floating point calculations if the triangle is very acute, i.e., 
    #if c is small relative to a and b or γ is small compared to 1. It is even possible to obtain a result slightly greater 
    #than one for the cosine of an angle.  #Cosine law: s4^2 = a2^2 + s3^2 - 2*as*s3*cos(beta2)
    #Set a limit over one to avoid imaginary numbers from the sqrt
    #if cosbeta2 > 1:
    #    cosbeta2 = 1
    if cosbeta2 > 1:
        cosbeta2 = 1
    beta2 = atan2(sqrt(1-pow(cosbeta2,2)), cosbeta2)
    #beta2 = acos((pow(1.25,2) + pow(s3,2) - pow(s4,2)) / (2 * 1.25 * s3))            
    theta2 = np.pi/2 - beta1 - beta2
            
    cos_angle_s4a2 = (pow(s4,2) + pow(1.25,2) - pow(s3,2)) / (2 * 1.25 * s4)
    if cos_angle_s4a2 > 1:
        cos_angle_s4a2 = 1                
    #angle_s4a2 = acos((pow(s4,2) + pow(1.25,2)-pow(s3,2))/(2 * 1.25 * s4))
    angle_s4a2 = atan2(sqrt(1-pow(cos_angle_s4a2,2)), cos_angle_s4a2)
    theta3 = np.pi/2 - 0.036 - angle_s4a2  
 
    #R3_G  =  (R0_3.inv("LU") * (R_rz * R_ry * R_rx)).subs({q1:theta1,q2:theta2,q3:theta3})
    # changing the method of inverting R0_3 had an impact on the calculations. Maybe try using ADJ rather than LU, 
    #or even use the transpose due to its equivalency with the inverse for that type of matrix
    R0_3 = R0_3.evalf(subs = ({q1:theta1,q2:theta2,q3:theta3}))           
    R3_G  =  (R0_3.transpose()) * R_rpy

    #compare the R3_G and R3_G_euler, we can get theta4,5,6
    if R3_G[2,2] > 1:
        R3_G[2,2] = 1
    #compare the R3_G and R3_G_euler, we can get theta4,5,6
    theta4 = atan2(R3_G[1,2],R3_G[0,2])
    theta5 = atan2(sqrt(1 - pow(R3_G[2,2],2)), R3_G[2,2]) - np.pi
    theta6 = atan2(-R3_G[2,1], R3_G[2,0])

   
    ########################################################################################
    # Populate response for the IK request
    print ("David THETA1=" + str(theta1))
    print ("David THETA2=" + str(theta2))
    print ("David THETA3=" + str(theta3))
    print ("David THETA4=" + str(theta4))
    print ("David THETA5=" + str(theta5))
    print ("David THETA6=" + str(theta6))
    ########################################################################################
    ## For additional debugging add your forward kinematics here. Use your previously calculated thetas
    ## as the input and output the position of your end effector as your_ee = [x,y,z]
    ## (OPTIONAL) YOUR CODE HERE!
    FK = T0_G.evalf(subs={q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6})
    ## End your code input for forward kinematics here!
    ########################################################################################

    ## For error analysis please set the following variables of your WC location and EE location in the format of [x,y,z]
    your_wc = [wx, wy, wz] # <--- Load your calculated WC values in this array
    your_ee = [FK[0,3],FK[1,3],FK[2,3]] # <--- Load your calculated end effector value from your forward kinematics
    ########################################################################################

    ## Error analysis
    print ("\nTest Case - Total run time to calculate joint angles from pose is %04.4f seconds" % (time()-start_time))

    # Find WC error
    if not(sum(your_wc)==3):
        wc_x_e = abs(your_wc[0]-test_case[1][0])
        wc_y_e = abs(your_wc[1]-test_case[1][1])
        wc_z_e = abs(your_wc[2]-test_case[1][2])
        wc_offset = sqrt(wc_x_e**2 + wc_y_e**2 + wc_z_e**2)
        print ("\nWrist error for x position is: %04.8f" % wc_x_e)
        print ("Wrist error for y position is: %04.8f" % wc_y_e)
        print ("Wrist error for z position is: %04.8f" % wc_z_e)
        print ("Overall wrist offset is: %04.8f units" % wc_offset)

    # Find theta errors
    t_1_e = abs(theta1-test_case[2][0])
    t_2_e = abs(theta2-test_case[2][1])
    t_3_e = abs(theta3-test_case[2][2])
    t_4_e = abs(theta4-test_case[2][3])
    t_5_e = abs(theta5-test_case[2][4])
    t_6_e = abs(theta6-test_case[2][5])
    print ("\nTheta 1 error is: %04.8f" % t_1_e)
    print ("Theta 2 error is: %04.8f" % t_2_e)
    print ("Theta 3 error is: %04.8f" % t_3_e)
    print ("Theta 4 error is: %04.8f" % t_4_e)
    print ("Theta 5 error is: %04.8f" % t_5_e)
    print ("Theta 6 error is: %04.8f" % t_6_e)
    print ("\n**These theta errors may not be a correct representation of your code, due to the fact \
           \nthat the arm can have muliple positions. It is best to add your forward kinmeatics to \
           \nconfirm whether your code is working or not**")
    print (" ")

    # Find FK EE error
    if not(sum(your_ee)==3):
        ee_x_e = abs(your_ee[0]-test_case[0][0][0])
        ee_y_e = abs(your_ee[1]-test_case[0][0][1])
        ee_z_e = abs(your_ee[2]-test_case[0][0][2])
        ee_offset = sqrt(ee_x_e**2 + ee_y_e**2 + ee_z_e**2)
        print ("\nEnd effector error for x position is: %04.8f" % ee_x_e)
        print ("End effector error for y position is: %04.8f" % ee_y_e)
        print ("End effector error for z position is: %04.8f" % ee_z_e)
        print ("Overall end effector offset is: %04.8f units \n" % ee_offset)




if __name__ == "__main__":
    # Change test case number for different scenarios
    for x in range(0, 3):
        print("Case " + str(x+1) + ": ")
        test_case_number = x+1
        test_code(test_cases[test_case_number])

Case 1: 
BEGIN INVERSE KINEMATICS CALCULATION
David THETA1=-0.669463299030768
David THETA2=0.795210573796331
David THETA3=-pi + 2.45522746126527
David THETA4=-0.912299704419874
David THETA5=-2.39682513667465
David THETA6=-pi + 1.01633978640692

Test Case - Total run time to calculate joint angles from pose is 0.5265 seconds

Wrist error for x position is: 0.24450419
Wrist error for y position is: 0.24975513
Wrist error for z position is: 0.28514545
Overall wrist offset is: 0.45107411 units

Theta 1 error is: 0.01946330
Theta 2 error is: 0.34521057
Theta 3 error is: 0.32636519
Theta 4 error is: 1.86229970
Theta 5 error is: 3.18682514
Theta 6 error is: 2.61525287

**These theta errors may not be a correct representation of your code, due to the fact            
that the arm can have muliple positions. It is best to add your forward kinmeatics to            
confirm whether your code is working or not**
 

End effector error for x position is: 0.08450946
End effector error for y position 